In [0]:
#importing libraries
import pandas as pd
import numpy as np
import random as random

In [8]:
#access to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#load dataset to a dataframe
df = pd.read_csv('/content/gdrive/My Drive/Genetic Algorithms/Assignment 01/dataset.csv')

In [0]:
#population = pd.DataFrame()
# function for init population
def createInitPop(popSize):
  chrom = [i for i in range(1,popSize+1)]
  #emp = [0 for i in range(0,popSize)]
  values ={
    'chromosome':chrom,
    'file 1':[random.randint(0, 1) for _ in range(popSize)],
    'file 2':[random.randint(0, 1) for _ in range(popSize)],
    'file 3':[random.randint(0, 1) for _ in range(popSize)],
    'file 4':[random.randint(0, 1) for _ in range(popSize)],
    'file 5':[random.randint(0, 1) for _ in range(popSize)],
    'file 6':[random.randint(0, 1) for _ in range(popSize)],
    'file 7':[random.randint(0, 1) for _ in range(popSize)],
    'file 8':[random.randint(0, 1) for _ in range(popSize)],
    'file 9':[random.randint(0, 1) for _ in range(popSize)],
    'file 10':[random.randint(0, 1) for _ in range(popSize)],
  }
  return values  

In [0]:
#Evaluate Fitness Function
def EvaluateFitness(chromosome,df,popSize):
  #fitness array
  fitness = {
      'chromosome':[x for x in range(1,popSize+1)],
      'fitness':[x for x in range(0,popSize)]
  }

  #convert to dataframe
  fitness = pd.DataFrame(fitness)

  #total file size
  totFileSize =[]

  #total file size and duration
  size=0
  duration=0

  #shape of the chromosome / population dataframe
  rows,col = chromosome.shape
  
  for i in range(rows):
    for j in range(1,11):
      if(chromosome.iloc[i,j] ==1):
        size += df.iloc[j-1,1]
        duration +=df.iloc[j-1,2]
    
    if(size<=4500):
      #fine
      fitness.loc[i,'fitness'] = duration
      totFileSize.append(size)
    else:
      fitness.loc[i,'fitness'] = duration*0.5
      totFileSize.append(size)

  return fitness, totFileSize      

In [0]:
#Rank Selection Function
def rankSelection(chromosome,fitness,popSize):
  
  rankTotal = sum(x for x in range(0,popSize))
  totRandom = rankTotal*random.uniform(0,rankTotal)
  tot=0
  fitness_sorted = fitness.sort_values(by=['fitness'])
  for i in fitness_sorted.index:
    tot += fitness_sorted.loc[i,'fitness']
    if(tot>=totRandom):
      fifo_index = i
      return chromosome.loc[i,:]
 

In [0]:
#crossover function
def doCrossoverMutate(chromosome,fitness,popSize):
  parent1 = rankSelection(chromosome,fitness,popSize)
  parent2 = rankSelection(chromosome,fitness,popSize)
  
  #random crossover point
  crossPoint = random.randint(1,popSize)
  
  c1p1 = parent1[1:crossPoint]
  c1p2 = parent1[crossPoint:11]
  
  child =c1p1.append(c1p2)
  
  child = bitflip(child)
  return child
  
  #return parent1, parent2
  #if(random.uniform(0,1)<0.75):
    #crossPoint = random.randint(1,popSize)
    #c1p1 = parent1.loc[,1:crossPoint]
    #c1p2 = parent2.loc[,crossPoint+1:11]
    
    #merge c1p1 and c1p2 as the new offspring
    
    
    

In [0]:
def bitflip(child):
  #generate random bit array
  bits = [random.uniform(0,1) for _ in range(10)]
  
  for i in range(10):
    if(bits[i]<0.1):
      if(child[i] == 0):
        child[i] = 1
      else:
        child[i] = 0
  
  return child
      

In [0]:
def replace_with_offspring(fifo_index,offSpring):
  chromosome.iloc[fifo_index,1:] = offSpring.values
  

In [0]:
#initialize
popSize = 16 #population size
fifo_index = -1
#creation of population of Chromosomes
chromosome = pd.DataFrame(createInitPop(popSize))

fitness, totFileSize = EvaluateFitness(chromosome,df,popSize)



In [461]:
chromosome

,chromosome,file 1,file 2,file 3,file 4,file 5,file 6,file 7,file 8,file 9,file 10
0,1,0,0,1,0,1,0,0,0,0,0
1,2,1,1,1,0,0,0,0,0,1,1
2,3,0,1,1,0,1,1,0,1,1,1
3,4,1,1,0,0,0,1,0,0,0,0
4,5,1,1,0,0,0,1,0,1,1,1
5,6,1,0,1,1,0,0,0,1,0,1
6,7,1,1,1,0,0,0,1,1,0,0
7,8,0,1,1,1,1,0,0,1,1,0
8,9,0,1,0,0,1,1,0,0,1,0
9,10,0,0,0,0,0,0,1,0,0,1


In [0]:
offSpring = doCrossoverMutate(chromosome,fitness,popSize)
replace_with_offspring(fifo_index,offSpring)

In [463]:
chromosome

,chromosome,file 1,file 2,file 3,file 4,file 5,file 6,file 7,file 8,file 9,file 10
0,1,0,0,1,0,1,0,0,0,0,0
1,2,1,1,1,0,0,0,0,0,1,1
2,3,0,1,1,0,1,1,0,1,1,1
3,4,1,1,0,0,0,1,0,0,0,0
4,5,1,1,0,0,0,1,0,1,1,1
5,6,1,0,1,1,0,0,0,1,0,1
6,7,1,1,1,0,0,0,1,1,0,0
7,8,0,1,1,1,1,0,0,1,1,0
8,9,0,1,0,0,1,1,0,0,1,0
9,10,0,0,0,0,0,0,1,0,0,1
